# Overfitting Analysis - FraudDetectAI

## Understanding Model Generalization

Overfitting occurs when a model **performs exceptionally well on training data** but fails to generalize to unseen data.  
Since our dataset is highly imbalanced and we applied **SMOTE**, there is a chance that some models may have **memorized synthetic data** rather than learning actual fraud patterns.

---

## **Objectives of This Notebook**
- Compare **training vs. test performance** to detect overfitting.  
- Visualize **learning curves** for potential divergence between train & validation loss.  
- Analyze **feature importance stability** (does SMOTE make the model over-rely on certain features?).  
- Decide whether to **adjust the SMOTE ratio** or introduce **undersampling**.

---

## **Steps We Will Cover**
1. **Compare Train vs. Test Performance** → Precision, Recall, F1-score, AUC-ROC  
2. **Plot Learning Curves** → Check whether the model is overfitting  
3. **Feature Importance Stability Check** → Compare SHAP values across datasets  
4. **Decision on SMOTE Adjustments** → Reduce synthetic samples? Add undersampling?  

**Imports**:

In [3]:
# Data Handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model Evaluation
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

# SHAP for Explainability
import shap

# Load Saved Models
import joblib

**Data Preparation**:

In [4]:
# Load normal dataset
X = pd.read_csv("../datasets/X_scaled.csv")
y = pd.read_csv("../datasets/y.csv")

# Load SMOTE dataset
X_smote = pd.read_csv("../datasets/X_smote.csv")
y_smote = pd.read_csv("../datasets/y_smote.csv")

# Perform Train-Test Split (for Base & Weighted models)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Perform Train-Test Split (for SMOTE model)
X_train_smote, _, y_train_smote, _ = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42, stratify=y_smote)

# X_test_smote should be the same as X_test (we don't oversample test data)
X_test_smote, y_test_smote = X_test, y_test

**Model Load**:

In [5]:
# Load trained models
base_xgb_opt = joblib.load("../models/optimized_base_xgb.pkl")
weighted_xgb_opt = joblib.load("../models/optimized_weighted_xgb.pkl")
smote_xgb_opt = joblib.load("../models/optimized_smote_xgb.pkl")

print("Models are loaded successfully.")

Models are loaded successfully.


## Comparing Training vs. Test Performance

To detect overfitting, we will compare how well our models perform on **training data vs. test data**.  
If a model **performs significantly better on training data** but struggles on the test set, it may be **overfitting**.

### **Steps for Evaluation**
1. **Make Predictions:**  
   - Predict on **X_train** and **X_test** for **Base & Weighted models**.  
   - Predict on **X_train_smote** and **X_test_smote** for **SMOTE model**.  

2. **Calculate Performance Metrics:**  
   - **Precision, Recall, F1-score, AUC-ROC** for both **train and test** sets.  

3. **Compare Results:**  
   - If **training performance is much higher** than test performance, overfitting is likely.  
   - If **metrics are consistent**, our model is generalizing well.  

**Function**:

In [6]:
# Define a function to evaluate model performance
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Compute metrics for training set
    train_precision = precision_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_auc = roc_auc_score(y_train, y_train_pred)

    # Compute metrics for test set
    test_precision = precision_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_auc = roc_auc_score(y_test, y_test_pred)
    
    # Print results
    print(f"\n{model_name} Performance:")
    print(f"Train - Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1-score: {train_f1:.4f}, AUC-ROC: {train_auc:.4f}")
    print(f"Test  - Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1-score: {test_f1:.4f}, AUC-ROC: {test_auc:.4f}")
    
    return {
        "Model": model_name,
        "Train Precision": train_precision,
        "Train Recall": train_recall,
        "Train F1-score": train_f1,
        "Train AUC-ROC": train_auc,
        "Test Precision": test_precision,
        "Test Recall": test_recall,
        "Test F1-score": test_f1,
        "Test AUC-ROC": test_auc,
    }

**Evaluation**:

In [7]:
# Evaluate Base XGBoost Model
base_results = evaluate_model(base_xgb_opt, X_train, y_train, X_test, y_test, "Base XGBoost")

# Evaluate Weighted XGBoost Model
weighted_results = evaluate_model(weighted_xgb_opt, X_train, y_train, X_test, y_test, "Weighted XGBoost")

# Evaluate SMOTE XGBoost Model
smote_results = evaluate_model(smote_xgb_opt, X_train_smote, y_train_smote, X_test_smote, y_test_smote, "SMOTE XGBoost")

# Store results for comparison
results_df = pd.DataFrame([base_results, weighted_results, smote_results])

# Display full dataframe output in Jupyter
pd.set_option("display.float_format", "{:.4f}".format)  # Format numbers
pd.set_option("display.max_columns", None)  # Show all columns

# Show the results table
display(results_df)

C:\Users\emrev\PycharmProjects\FraudDetectAI\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [23:37:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)



Base XGBoost Performance:
Train - Precision: 1.0000, Recall: 1.0000, F1-score: 1.0000, AUC-ROC: 1.0000
Test  - Precision: 0.8925, Recall: 0.8469, F1-score: 0.8691, AUC-ROC: 0.9234

Weighted XGBoost Performance:
Train - Precision: 0.9975, Recall: 1.0000, F1-score: 0.9987, AUC-ROC: 1.0000
Test  - Precision: 0.8617, Recall: 0.8265, F1-score: 0.8438, AUC-ROC: 0.9132

SMOTE XGBoost Performance:
Train - Precision: 0.9998, Recall: 1.0000, F1-score: 0.9999, AUC-ROC: 0.9999
Test  - Precision: 0.5833, Recall: 1.0000, F1-score: 0.7368, AUC-ROC: 0.9994


,Model,Train Precision,Train Recall,Train F1-score,Train AUC-ROC,Test Precision,Test Recall,Test F1-score,Test AUC-ROC
0,Base XGBoost,1.0000,1.0000,1.0000,1.0000,0.8925,0.8469,0.8691,0.9234
1,Weighted XGBoost,0.9975,1.0000,0.9987,1.0000,0.8617,0.8265,0.8438,0.9132
2,SMOTE XGBoost,0.9998,1.0000,0.9999,0.9999,0.5833,1.0000,0.7368,0.9994


## Overfitting Analysis - Train vs. Test Performance  

After evaluating model performance on **both training and test sets**, we found some key insights about potential overfitting.  

---

### **Key Observations:**  
1. **Base XGBoost:**  
   - **Train F1-score = 1.0000 (Perfect prediction)** → **Possible overfitting**  
   - **Test F1-score = 0.8691**, AUC-ROC = 0.9234 → **Decent generalization** but **train is too high**.  

2. **Weighted XGBoost:**  
   - **Train F1-score = 0.9987 (Almost perfect)** → **Slight overfitting**  
   - **Test F1-score = 0.8438**, AUC-ROC = 0.9132 → **Similar to Base XGBoost** but penalizing fraud slightly improved recall.  

3. **SMOTE XGBoost:** 
   - **Train Recall = 1.0000, Precision = 0.9998** → **Overfitting confirmed**  
   - **Test Precision = 0.5833**, **Test Recall = 1.0000** → **It’s catching all frauds but also classifying too many non-frauds as fraud!**  
   - **Train AUC-ROC = 0.9999 vs. Test AUC-ROC = 0.9994** → **This suggests it memorized synthetic patterns but struggles on real data.**  

---

### **What Should We Do Next?**  
1. **Step 1: Visualize Learning Curves**  
   - Plot **Train vs. Test Loss** to **confirm overfitting trends.**  
2. **Step 2: Adjust SMOTE Ratio (If Needed)**  
   - Instead of **1:1 oversampling**, try **7:3 or 8:2** to keep it more balanced.  
3. **Step 3: Consider Undersampling**  
   - Combine **SMOTE with undersampling** to **remove unnecessary non-fraud cases.**  

---

### **Immediate Next Step: Plot Learning Curves**  
We will now **visualize training vs. validation loss curves** for all models to **confirm overfitting trends**.